# Human Connectome Project (HCP) Dataset loader. Saves out individual subject CSV files.

The HCP dataset comprises resting-state and task-based fMRI from a large sample of human subjects. The NMA-curated dataset includes time series data that has been preprocessed and spatially-downsampled by aggregating within 360 regions of interest.

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [3]:
#@title Figure settings
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

# Basic parameters

In [4]:
# The download cells will store the data in nested directories starting here:
HCP_DIR = "./hcp"
if not os.path.isdir(HCP_DIR):
  os.mkdir(HCP_DIR)

# The data shared for NMA projects is a subset of the full HCP dataset
N_SUBJECTS = 339

# The data have already been aggregated into ROIs from the Glasesr parcellation
N_PARCELS = 360

# The acquisition parameters for all tasks were identical
TR = 0.72  # Time resolution, in sec

# The parcels are matched across hemispheres with the same order
HEMIS = ["Right", "Left"]

# Each experiment was repeated multiple times in each subject
N_RUNS_REST = 4
N_RUNS_TASK = 2

# Time series data are organized by experiment, with each experiment
# having an LR and RL (phase-encode direction) acquistion
BOLD_NAMES = [
  "rfMRI_REST1_LR", "rfMRI_REST1_RL",
  "rfMRI_REST2_LR", "rfMRI_REST2_RL",
  "tfMRI_MOTOR_RL", "tfMRI_MOTOR_LR",
  "tfMRI_WM_RL", "tfMRI_WM_LR",
  "tfMRI_EMOTION_RL", "tfMRI_EMOTION_LR",
  "tfMRI_GAMBLING_RL", "tfMRI_GAMBLING_LR",
  "tfMRI_LANGUAGE_RL", "tfMRI_LANGUAGE_LR",
  "tfMRI_RELATIONAL_RL", "tfMRI_RELATIONAL_LR",
  "tfMRI_SOCIAL_RL", "tfMRI_SOCIAL_LR"
]

# You may want to limit the subjects used during code development.
# This will use all subjects:
subjects = range(N_SUBJECTS)

# Downloading data

The rest and task data are shared in different files, but they will unpack into the same directory structure.

Each file is fairly large and will take some time to download. If you are focusing only on rest or task analyses, you may not want to download only that dataset.

In [5]:
fname = "hcp_rest.tgz"
import wget
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/bqp7m/download/
  !tar -xzf $fname -C $HCP_DIR --strip-components=1

In [6]:
fname = "hcp_task.tgz"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/s4h8j/download/
  !tar -xzf $fname -C $HCP_DIR --strip-components=1

## Loading region information

Downloading either dataset will create the `regions.npy` file, which contains the region name and network assignment for each parcel.

Detailed information about the name used for each region is provided [in the Supplement](https://static-content.springer.com/esm/art%3A10.1038%2Fnature18933/MediaObjects/41586_2016_BFnature18933_MOESM330_ESM.pdf) to [Glasser et al. 2016](https://www.nature.com/articles/nature18933).

Information about the network parcellation is provided in [Ji et al, 2019](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6289683/).

In [7]:
regions = np.load(f"{HCP_DIR}/regions.npy").T
region_info = dict(
    name=regions[0].tolist(),
    network=regions[1],
    myelin=regions[2].astype(np.float),
)

# Helper functions


## Data loading

In [8]:
def get_image_ids(name):
  """Get the 1-based image indices for runs in a given experiment.

    Args:
      name (str) : Name of experiment ("rest" or name of task) to load
    Returns:
      run_ids (list of int) : Numeric ID for experiment image files

  """
  run_ids = [
    i for i, code in enumerate(BOLD_NAMES, 1) if name.upper() in code
  ]
  if not run_ids:
    raise ValueError(f"Found no data for '{name}''")
  return run_ids

def load_timeseries(subject, name, runs=None, concat=True, remove_mean=True):
  """Load timeseries data for a single subject.
  
  Args:
    subject (int): 0-based subject ID to load
    name (str) : Name of experiment ("rest" or name of task) to load
    run (None or int or list of ints): 0-based run(s) of the task to load,
      or None to load all runs.
    concat (bool) : If True, concatenate multiple runs in time
    remove_mean (bool) : If True, subtract the parcel-wise mean

  Returns
    ts (n_parcel x n_tp array): Array of BOLD data values

  """
  # Get the list relative 0-based index of runs to use
  if runs is None:
    runs = range(N_RUNS_REST) if name == "rest" else range(N_RUNS_TASK)
  elif isinstance(runs, int):
    runs = [runs]

  # Get the first (1-based) run id for this experiment 
  offset = get_image_ids(name)[0]

  # Load each run's data
  bold_data = [
      load_single_timeseries(subject, offset + run, remove_mean) for run in runs
  ]

  # Optionally concatenate in time
  if concat:
    bold_data = np.concatenate(bold_data, axis=-1)

  return bold_data


def load_single_timeseries(subject, bold_run, remove_mean=True):
  """Load timeseries data for a single subject and single run.
  
  Args:
    subject (int): 0-based subject ID to load
    bold_run (int): 1-based run index, across all tasks
    remove_mean (bool): If True, subtract the parcel-wise mean

  Returns
    ts (n_parcel x n_timepoint array): Array of BOLD data values

  """
  bold_path = f"{HCP_DIR}/subjects/{subject}/timeseries"
  bold_file = f"bold{bold_run}_Atlas_MSMAll_Glasser360Cortical.npy"
  ts = np.load(f"{bold_path}/{bold_file}")
  if remove_mean:
    ts -= ts.mean(axis=1, keepdims=True)
  return ts

def load_evs(subject, name, condition):
  """Load EV (explanatory variable) data for one task condition.

  Args:
    subject (int): 0-based subject ID to load
    name (str) : Name of task
    condition (str) : Name of condition

  Returns
    evs (list of dicts): A dictionary with the onset, duration, and amplitude
      of the condition for each run.

  """
  evs = []
  #Check if file is empty

        
  for id in get_image_ids(name):
    task_key = BOLD_NAMES[id - 1]
    ev_file = f"{HCP_DIR}/subjects/{subject}/EVs/{task_key}/{condition}.txt"
    if os.stat(ev_file).st_size != 0:
        ev_array = np.loadtxt(ev_file, ndmin=2, unpack=True)
        ev = dict(zip(["onset", "duration", "amplitude"], ev_array))
        evs.append(ev)
    else:
        evs.append(dict(zip(["onset", "duration", "amplitude"], np.array([[],[],[]]))))
  return evs

## Task-based analysis

In [9]:
def condition_frames(run_evs, skip=0):
  """Identify timepoints corresponding to a given condition in each run.

  Args:
    run_evs (list of dicts) : Onset and duration of the event, per run
    skip (int) : Ignore this many frames at the start of each trial, to account
      for hemodynamic lag

  Returns:
    frames_list (list of 1D arrays): Flat arrays of frame indices, per run

  """
  frames_list = []

  for ev in run_evs:
    
    if np.shape(ev['onset'])[0]!=0:

        # Determine when trial starts, rounded down
        start = np.floor(ev["onset"] / TR).astype(int)

        # Use trial duration to determine how many frames to include for trial
        duration = np.ceil(ev["duration"] / TR).astype(int)

        # Take the range of frames that correspond to this specific trial
        frames = [s + np.arange(skip, d) for s, d in zip(start, duration)]

        frames_list.append(np.concatenate(frames))
    
    else:
        frames_list.append([])


  return frames_list


def selective_average(timeseries_data, ev, skip=0):
  """Take the temporal mean across frames for a given condition.

  Args:
    timeseries_data (array or list of arrays): n_parcel x n_tp arrays
    ev (dict or list of dicts): Condition timing information
    skip (int) : Ignore this many frames at the start of each trial, to account
      for hemodynamic lag

  Returns:
    avg_data (1D array): Data averagted across selected image frames based
    on condition timing

  """
  # Ensure that we have lists of the same length
  if not isinstance(timeseries_data, list):
    timeseries_data = [timeseries_data]
  if not isinstance(ev, list):
    ev = [ev]
  if len(timeseries_data) != len(ev):
    raise ValueError("Length of `timeseries_data` and `ev` must match.")

  # Identify the indices of relevant frames
  frames = condition_frames(ev)

  # Select the frames from each image
  selected_data = []
  for run_data, run_frames in zip(timeseries_data, frames):
    selected_data.append(run_data[:, run_frames])

  # Take the average in each parcel
  avg_data = np.concatenate(selected_data, axis=-1).mean(axis=-1)

  return avg_data

## Create all-task CSVs for each subject

In [10]:
task_names = ['rest', 'motor', 'wm', 'emotion', 'gambling', 'language', 'relational', 'social']
task_nos = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
columns = np.hstack((np.array(['subject', 'task_name', 'task_num']),region_info['name']))
pd_subj = pd.DataFrame(columns=columns)

for subj in subjects:
    pd_subj = pd.DataFrame(columns=columns)
    for i_t, t in enumerate(task_names):
        # homework: figure out how to append trial number too
        # use t for task name, i_t to get integer for task number
        data = load_timeseries(subj, t, concat=True).T
        n_trials = data.shape[0]
        info = np.tile(np.array([subj, t, i_t]), n_trials).reshape(n_trials, -1)
        data_stacked = np.hstack((info, data))
        df_task = pd.DataFrame(data_stacked, columns = columns)
        pd_subj = pd_subj.append(df_task, ignore_index=True)
    pd_subj.to_csv('%i_all_task.csv'%subj)

KeyboardInterrupt: 

## Make subject dataframes and CSVs for boredom analysis

In [11]:
task_names = ['rest', 'wm']
task_nos = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
columns = np.hstack((np.array(['subject', 'task_name', 'task_num', 'run', 'condition', 'frame_num','trial_num', 'correct_or_not', 'classifier_correct', 'classifer_prediction', 'predict_prob_WM', 'predict_prob_rest']), region_info['name']))
pd_subj = pd.DataFrame(columns=columns)

for subj in subjects:
    pd_subj = pd.DataFrame(columns=columns)
    for i_t, t in enumerate(task_names):
        if t == 'wm':
            for run in range(2):
                data = load_timeseries(subj, t, runs=run, concat=True).T
                n_frames = data.shape[0]
                info = np.tile(np.array([subj, t, i_t, run+1, 0, 0, 0, 0, 0, 0, 0, 0 ]), n_frames).reshape(n_frames, -1)
                data_stacked = np.hstack((info, data))
                df_task = pd.DataFrame(data_stacked, columns = columns)
                df_task['frame_num'] = range(len(df_task))
                pd_subj = pd_subj.append(df_task, ignore_index=True)
#                 pd_subj.loc[(pd_subj['run']==run+1)&(pd_subj['task_name']=='wm'), 'frame_num'] = range(len(pd_subj.loc[(pd_subj['run']==run+1)&(pd_subj['task_name']=='wm')]))
        elif t =='rest':
            data = load_timeseries(subj, t, concat=True).T
            n_frames = data.shape[0]
            info = np.tile(np.array([subj, t, i_t, 0, 0, 0, 0, 0, 0, 0, 0, 0 ]), n_frames).reshape(n_frames, -1)
            data_stacked = np.hstack((info, data))
            df_task = pd.DataFrame(data_stacked, columns = columns)
            pd_subj = pd_subj.append(df_task, ignore_index=True)
            pd_subj.loc[pd_subj['task_name']=='rest', 'frame_num'] = range(len(pd_subj.loc[pd_subj['task_name']=='rest']))
        
        if t == 'wm':
            bk_0_cor = load_evs(subj,'wm','0bk_cor')
            bk_0_cor_frame_indices_run1 = condition_frames(bk_0_cor)[0]
            bk_0_cor_frame_indices_run2 = condition_frames(bk_0_cor)[1]

            bk_0_err = load_evs(subj,'wm','0bk_err')
            bk_0_err_frame_indices_run1 = condition_frames(bk_0_err)[0]
            bk_0_err_frame_indices_run2 = condition_frames(bk_0_err)[1]

            bk_0_nlr = load_evs(subj,'wm','0bk_nlr')
            bk_0_nlr_frame_indices_run1 = condition_frames(bk_0_nlr)[0]
            bk_0_nlr_frame_indices_run2 = condition_frames(bk_0_nlr)[1]

            bk_2_cor = load_evs(subj,'wm','2bk_cor')
            bk_2_cor_frame_indices_run1 = condition_frames(bk_2_cor)[0]  
            bk_2_cor_frame_indices_run2 = condition_frames(bk_2_cor)[1]         

            bk_2_err = load_evs(subj,'wm','2bk_err')
            bk_2_err_frame_indices_run1 = condition_frames(bk_2_err)[0]    
            bk_2_err_frame_indices_run2 = condition_frames(bk_2_err)[1]         

            bk_2_nlr = load_evs(subj,'wm','2bk_nlr')
            bk_2_nlr_frame_indices_run1 = condition_frames(bk_2_nlr)[0]
            bk_2_nlr_frame_indices_run2 = condition_frames(bk_2_nlr)[1]

    
    pd_subj.loc[(pd_subj.run=='1') & pd_subj.frame_num.isin(bk_0_cor_frame_indices_run1), 'condition'] = '0_bk'
    pd_subj.loc[(pd_subj.run=='1') & pd_subj.frame_num.isin(bk_0_cor_frame_indices_run1), 'correct_or_not'] = 'cor'

    pd_subj.loc[(pd_subj.run=='1') & pd_subj.frame_num.isin(bk_0_err_frame_indices_run1), 'condition'] = '0_bk'
    pd_subj.loc[(pd_subj.run=='1') & pd_subj.frame_num.isin(bk_0_err_frame_indices_run1) & (pd_subj.correct_or_not != 'cor'), 'correct_or_not'] = 'err'

    pd_subj.loc[(pd_subj.run=='1') & pd_subj.frame_num.isin(bk_0_nlr_frame_indices_run1), 'condition'] = '0_bk'
    pd_subj.loc[(pd_subj.run=='1') & pd_subj.frame_num.isin(bk_0_nlr_frame_indices_run1) & (pd_subj.correct_or_not != 'cor') & (pd_subj.correct_or_not != 'nlr'), 'correct_or_not'] = 'nlr'

    pd_subj.loc[(pd_subj.run=='2') & pd_subj.frame_num.isin(bk_0_cor_frame_indices_run2), 'condition'] = '0_bk'
    pd_subj.loc[(pd_subj.run=='2') & pd_subj.frame_num.isin(bk_0_cor_frame_indices_run2), 'correct_or_not'] = 'cor'

    pd_subj.loc[(pd_subj.run=='2') & pd_subj.frame_num.isin(bk_0_err_frame_indices_run2), 'condition'] = '0_bk'
    pd_subj.loc[(pd_subj.run=='2') & pd_subj.frame_num.isin(bk_0_err_frame_indices_run2) & (pd_subj.correct_or_not != 'cor'), 'correct_or_not'] = 'err'
    
    pd_subj.loc[(pd_subj.run=='2') & pd_subj.frame_num.isin(bk_0_nlr_frame_indices_run2), 'condition'] = '0_bk'
    pd_subj.loc[(pd_subj.run=='2') & pd_subj.frame_num.isin(bk_0_nlr_frame_indices_run2) & (pd_subj.correct_or_not != 'cor') & (pd_subj.correct_or_not != 'nlr'), 'correct_or_not'] = 'nlr'

    
    pd_subj.loc[(pd_subj.run=='1') & pd_subj.frame_num.isin(bk_2_cor_frame_indices_run1), 'condition'] = '2_bk'
    pd_subj.loc[(pd_subj.run=='1') & pd_subj.frame_num.isin(bk_2_cor_frame_indices_run1), 'correct_or_not'] = 'cor'

    pd_subj.loc[(pd_subj.run=='1') & pd_subj.frame_num.isin(bk_2_err_frame_indices_run1), 'condition'] = '2_bk'
    pd_subj.loc[(pd_subj.run=='1') & pd_subj.frame_num.isin(bk_2_err_frame_indices_run1) & (pd_subj.correct_or_not != 'cor'), 'correct_or_not'] = 'err'

    pd_subj.loc[(pd_subj.run=='1') & pd_subj.frame_num.isin(bk_2_nlr_frame_indices_run1), 'condition'] = '2_bk'
    pd_subj.loc[(pd_subj.run=='1') & pd_subj.frame_num.isin(bk_2_nlr_frame_indices_run1) & (pd_subj.correct_or_not != 'cor') & (pd_subj.correct_or_not != 'nlr'), 'correct_or_not'] = 'nlr'

    pd_subj.loc[(pd_subj.run=='2') & pd_subj.frame_num.isin(bk_2_cor_frame_indices_run2), 'condition'] = '2_bk'
    pd_subj.loc[(pd_subj.run=='2') & pd_subj.frame_num.isin(bk_2_cor_frame_indices_run2), 'correct_or_not'] = 'cor'

    pd_subj.loc[(pd_subj.run=='2') & pd_subj.frame_num.isin(bk_2_err_frame_indices_run2), 'condition'] = '2_bk'
    pd_subj.loc[(pd_subj.run=='2') & pd_subj.frame_num.isin(bk_2_err_frame_indices_run2) & (pd_subj.correct_or_not != 'cor'), 'correct_or_not'] = 'err'

    pd_subj.loc[(pd_subj.run=='2') & pd_subj.frame_num.isin(bk_2_nlr_frame_indices_run2), 'condition'] = '2_bk'
    pd_subj.loc[(pd_subj.run=='2') & pd_subj.frame_num.isin(bk_2_nlr_frame_indices_run2) & (pd_subj.correct_or_not != 'cor') & (pd_subj.correct_or_not != 'nlr'), 'correct_or_not'] = 'nlr'
    
    sorted_onsets = [[],[]]
    sorted_durations = [[],[]]
    
    sorted_onsets[0] = np.sort(np.hstack((bk_0_cor[0]['onset'], bk_0_err[0]['onset'], bk_0_nlr[0]['onset'], bk_2_cor[0]['onset'], bk_2_err[0]['onset'], bk_2_nlr[0]['onset'])))
    sorted_durations[0] = np.sort(np.hstack((bk_0_cor[0]['duration'], bk_0_err[0]['duration'], bk_0_nlr[0]['duration'], bk_2_cor[0]['duration'], bk_2_err[0]['duration'], bk_2_nlr[0]['duration'])))

    sorted_onsets[1] = np.sort(np.hstack((bk_0_cor[1]['onset'], bk_0_err[1]['onset'], bk_0_nlr[1]['onset'], bk_2_cor[1]['onset'], bk_2_err[1]['onset'], bk_2_nlr[1]['onset'])))
    sorted_durations[1] = np.sort(np.hstack((bk_0_cor[1]['duration'], bk_0_err[1]['duration'], bk_0_nlr[1]['duration'], bk_2_cor[1]['duration'], bk_2_err[1]['duration'], bk_2_nlr[1]['duration'])))
    
    for run in range(2):
        
        frames_list = []

        # Determine when trial starts, rounded down
        start = np.floor(sorted_onsets[run] / TR).astype(int)

        # Use trial duration to determine how many frames to include for trial
        duration = np.ceil(sorted_durations[run] / TR).astype(int)

        # Take the range of frames that correspond to this specific trial
        frames = [s + np.arange(0, d) for s, d in zip(start, duration)]

        frames_list.append(np.concatenate(frames))

        for trial in range(len(sorted_onsets[run])):
            
            pd_subj.loc[(pd_subj.run==str(run+1))&(pd_subj.frame_num.isin(frames_list[0][trial*4:(trial+1)*4])), 'trial_num'] = trial+1

        
    pd_subj.to_csv('%i_boredom_df.csv'%subj)